In [9]:
import cv2
import numpy as np
import os
import pandas as pd
from scipy.stats import entropy, pearsonr

# import otsu from your src folder
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))
from otsu import otsu_threshold


# ===== Utility functions =====

def global_hist_equalize(gray_img):
    """Apply global histogram equalization."""
    return cv2.equalizeHist(gray_img)

def calculate_histogram_entropy(img):
    """Entropy of the 256‑bin grayscale histogram (bits)."""
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    hist_norm = hist.ravel() / (hist.sum() + 1e-9)
    return entropy(hist_norm, base=2)

def calculate_mean_contrast(img):
    """Global contrast = std of pixel intensities."""
    return float(np.std(img))

def calculate_pixel_proportions(binary_img):
    """% white/black in a binary mask (0/255)."""
    total = binary_img.size
    white = int(np.sum(binary_img == 255))
    black = total - white
    return (white / total * 100.0, black / total * 100.0)

def laplacian_variance(gray):
    """Focus/sharpness: variance of Laplacian."""
    return float(cv2.Laplacian(gray, cv2.CV_64F).var())

def sobel_magnitude(gray):
    """Sobel gradient magnitude (float32)."""
    sx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    sy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    mag = cv2.magnitude(sx, sy)
    return mag

def edge_density_and_preservation(raw_gray, proc_gray):
    """
    Edge density on processed image + edge preservation vs raw.
    Density: proportion of pixels above Otsu threshold on Sobel magnitude.
    Preservation: Pearson r between raw and processed Sobel magnitudes.
    """
    mag_raw = sobel_magnitude(raw_gray)
    mag_proc = sobel_magnitude(proc_gray)

    # Edge density (auto threshold on processed magnitude)
    # Normalize to 8-bit for Otsu
    m = mag_proc
    m8 = cv2.normalize(m, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    _, edges_bin = cv2.threshold(m8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    edge_density = float((edges_bin > 0).mean() * 100.0)

    # Edge preservation (Pearson correlation of magnitudes)
    # Flatten and guard against zero variance
    v1 = mag_raw.ravel()
    v2 = mag_proc.ravel()
    if np.std(v1) < 1e-9 or np.std(v2) < 1e-9:
        preservation_r = np.nan
    else:
        preservation_r, _ = pearsonr(v1, v2)
        preservation_r = float(preservation_r)

    return edge_density, preservation_r

def illumination_uniformity(bgr_or_gray):
    """
    Illumination uniformity on L channel:
    compute a heavily blurred illumination field and report std/mean (%).
    Lower is better (more uniform).
    """
    if len(bgr_or_gray.shape) == 2:
        bgr = cv2.cvtColor(bgr_or_gray, cv2.COLOR_GRAY2BGR)
    else:
        bgr = bgr_or_gray

    L = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)[:, :, 0].astype(np.float32)
    illum = cv2.GaussianBlur(L, (0, 0), sigmaX=45, sigmaY=45)
    unif = float((illum.std() / (illum.mean() + 1e-9)) * 100.0)  # %
    return unif

def roi_center_offset_and_local_contrast(proc_gray, binary_mask):
    """
    ROI centering: distance of foreground centroid to image center (as % of diagonal).
    ROI local contrast: std of intensities within the white mask.
    """
    mask = (binary_mask == 255).astype(np.uint8)
    h, w = mask.shape
    M = cv2.moments(mask, binaryImage=True)
    if M["m00"] == 0:
        return np.nan, np.nan  # no ROI

    cx = M["m10"] / M["m00"]
    cy = M["m01"] / M["m00"]
    # center distance normalized by half-diagonal -> % of diagonal
    dx = cx - (w / 2.0)
    dy = cy - (h / 2.0)
    diag = np.sqrt(w**2 + h**2)
    center_offset_pct = float((np.sqrt(dx*dx + dy*dy) / (diag / 2.0)) * 100.0)

    # local contrast inside ROI
    roi_vals = proc_gray[mask > 0]
    if roi_vals.size == 0:
        roi_contrast = np.nan
    else:
        roi_contrast = float(np.std(roi_vals))

    return center_offset_pct, roi_contrast


# ===== Core image processing =====

def process_image(filename, input_folder, otsu_function):
    """Process a single image and return metrics."""
    path = os.path.join(input_folder, filename)
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)  # keep original channels
    if img is None:
        raise ValueError(f"Image not found: {path}")

    # raw grayscale
    if len(img.shape) == 3 and img.shape[2] == 3:
        gray_raw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        bgr_for_lab = img
    else:
        gray_raw = img
        bgr_for_lab = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    # equalized grayscale (for most metrics)
    gray_eq = global_hist_equalize(gray_raw)

    # core metrics
    entropy_val = calculate_histogram_entropy(gray_eq)
    contrast_val = calculate_mean_contrast(gray_eq)
    sharpness_lap = laplacian_variance(gray_eq)
    edge_density_pct, edge_preservation_r = edge_density_and_preservation(gray_raw, gray_eq)
    illum_unif_pct = illumination_uniformity(bgr_for_lab)

    # Otsu expects BGR; feed equalized as BGR
    gray_eq_bgr = cv2.cvtColor(gray_eq, cv2.COLOR_GRAY2BGR)
    binary_img, _ = otsu_function(gray_eq_bgr, fname=filename)

    white_pct, black_pct = calculate_pixel_proportions(binary_img)
    center_offset_pct, roi_contrast = roi_center_offset_and_local_contrast(gray_eq, binary_img)

    return {
        'Filename': filename,
        'Histogram Entropy': entropy_val,
        'Mean Contrast': contrast_val,
        'Sharpness (Laplacian Var)': sharpness_lap,
        'Edge Density (%)': edge_density_pct,
        'Edge Preservation (r)': edge_preservation_r,
        'Illumination Uniformity (%)': illum_unif_pct,
        '% White Pixels': white_pct,
        '% Black Pixels': black_pct,
        'ROI Center Offset (%)': center_offset_pct,
        'ROI Local Contrast': roi_contrast,
    }


# ===== Batch processing =====

def batch_process_images(input_folder='../data/test_images',
                         output_file='image_quality_metrics_pipeline13.xlsx',
                         otsu_function=None):
    """Process all images in a folder and export results to Excel."""
    assert otsu_function is not None, "Please pass your Otsu thresholding function."

    results = []
    for filename in sorted(os.listdir(input_folder)):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
            try:
                res = process_image(filename, input_folder, otsu_function)
                results.append(res)
            except Exception as e:
                print(f"Skipping {filename}: {e}")

    df = pd.DataFrame(results)
    df.to_excel(output_file, index=False)
    print(f"✅ Saved metrics to {output_file} with {len(df)} rows")
    return df


# ===== Run the analysis =====

df = batch_process_images(
    input_folder="../data/test_images",
    output_file="/Users/sydneysmith/Desktop/Pipeline_Images/metric_tests/image_quality_metrics_pipeline13.xlsx",
    otsu_function=otsu_threshold
)
print(df.head())


✅ Saved metrics to /Users/sydneysmith/Desktop/Pipeline_Images/metric_tests/image_quality_metrics_pipeline13.xlsx with 147 rows
                                            Filename  Histogram Entropy  \
0  T0004-04-06-2019_BL (2)_processed_piplinetest1...           6.590774   
1     T0011-06-06-2019_N_processed_piplinetest13.JPG           7.293568   
2  T0013-06-06-2019_BL (1)_processed_piplinetest1...           6.938532   
3     T0013-06-06-2019_N_processed_piplinetest13.JPG           6.756797   
4   T0017_10-06-2019 (1)_processed_piplinetest13.JPG           7.176872   

   Mean Contrast  Sharpness (Laplacian Var)  Edge Density (%)  \
0      73.700534               10116.671822         17.964764   
1      73.467230               20749.935345         19.507334   
2      73.275684                7601.517168         24.178890   
3      73.081921                8423.327606         23.676658   
4      73.592581               21295.240556         19.282127   

   Edge Preservation (r)  Illum

In [5]:
%pip install pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 4.3 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [openpyxl]4/5 [openpyxl]
Note: you may need to restart the kernel to use updated packages.
